In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/dataset-2/train.csv')  
print(data.head())


          src_ipv4                                 src_ipv6  \
0    171.153.21.53  28f9:dadc:4971:c47e:5ab6:b29a:d7e9:be69   
1       66.7.16.37    5191:bb30:19:3d6c:1bfb:6ce0:91c6:34a4   
2  172.153.167.203  2663:e398:17eb:d8cf:de62:1bf3:3c65:8e9a   
3    76.170.201.34   a6f3:d510:8145:da93:c305:706:8aa7:35e0   
4   160.33.148.177  f310:1792:cd5a:8d7d:82db:9b51:1e92:3de4   

             src_mac  src_port  dest_port  duration       os  packet_size tcp  \
0  e5:d4:16:af:cd:3d     38994      43061    197885  windows       340182   t   
1  88:83:e8:c7:72:6f     26001      60218   1003599  windows       357240   f   
2  4f:38:5b:85:8f:6f     14730      15793    496310  windows       631837   f   
3  a2:b4:03:92:e4:02      2417       5858    149950    macos       809215   t   
4  93:63:c1:f6:2d:28     42082      41166    708751  windows       133730   t   

  udp  ... ftp icmp syn ack fin psh urg ece cwr attack_type  
0   t  ...   f    t   t   t   t   f   t   t   f       Mirai  
1   t  ...

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Handle missing values 
data.fillna(0, inplace=True)  

# Encode categorical variables
label_encoders = {}
categorical_features = ['src_ipv4', 'src_ipv6', 'src_mac', 'os', 'tcp', 'udp', 'http', 
                        'https', 'ssh', 'smtp', 'pop3', 'ftp', 'icmp', 'syn', 'ack', 
                        'fin', 'psh', 'urg', 'ece', 'cwr', 'attack_type']

for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature])
    label_encoders[feature] = le  
    
# Define features (X) and target (y)
X = data.drop('attack_type', axis=1)
y = data['attack_type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))


Accuracy: 0.20
              precision    recall  f1-score   support

           0       0.20      0.23      0.21     22546
           1       0.20      0.20      0.20     22410
           2       0.20      0.19      0.19     22395
           3       0.20      0.19      0.19     22193
           4       0.20      0.18      0.19     22456

    accuracy                           0.20    112000
   macro avg       0.20      0.20      0.20    112000
weighted avg       0.20      0.20      0.20    112000

